The first step in creating a neural network model is to define the Input layer. This layer takes in raw data, usually in the form of numpy arrays. The shape of the Input layer defines how many variables your neural network will use. For example, if the input data has 10 columns, you define an Input layer with a shape of (10,)

In [1]:
# Import Input from keras.layers
from keras.layers import Input

# Create an input layer of shape 1
input_tensor = Input(shape=(1,))

C:\Users\Gaya\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load layers
from keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Dense layer
output_layer = Dense(1)

# Connect the dense layer to the input_tensor
output_tensor = output_layer(input_tensor)

In [3]:
# Load layers
from keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Create a dense layer and connect the dense layer to the input_tensor in one step
# Note that we did this in 2 steps in the previous exercise, but are doing it in one step now
output_tensor = Dense(1)(input_tensor)

In [4]:
# Input/dense/output layers
from keras.layers import Input, Dense
input_tensor = Input(shape=(1,))
output_tensor = Dense(1)(input_tensor)

# Build the model
from keras.models import Model
model = Model(input_tensor, output_tensor)

The final step in creating a model is compiling it. Now that you've created a model, you have to compile it before you can fit it to data. This finalizes your model, freezes all its settings, and prepares it to meet some data!

During compilation, you specify the optimizer to use for fitting the model to the data, and a loss function. 'adam' is a good default optimizer to use, and will generally work well. Loss function depends on the problem at hand. Mean squared error is a common loss function and will optimize for predicting the mean, as is done in least squares regression.

Mean absolute error optimizes for the median and is used in quantile regression. For this dataset, 'mean_absolute_error' works pretty well, so use it as your loss function.

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

Visualize a model
Now that you've compiled the model, take a look a the result of your hard work! You can do this by looking at the model summary, as well as its plot.

The summary will tell you the names of the layers, as well as how many units they have and how many parameters are in the model.

The plot will show how the layers connect to each other.

In [6]:
import os
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\release\\bin\\'

In [7]:
# Import the plotting function
import pydot
from keras.utils import plot_model
import matplotlib.pyplot as plt

# Summarize the model
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
games_tourney=pd.read_csv('E:\\PYTHON\\Data\\basketball\\games_tourney.csv')

In [9]:
games_tourney_train,games_tourney_test=games_tourney.loc[:3430,:],games_tourney.loc[3430:,:]

In [10]:
# Now fit the model
model.fit(games_tourney_train['seed_diff'], games_tourney_train['score_diff'],
          epochs=1,
          batch_size=128,
          validation_split=0.1,
          verbose=True)

Train on 3087 samples, validate on 344 samples
Epoch 1/1
3087/3087 [==============================] - 0s 74us/step - loss: 9.2800 - val_loss: 8.5836


In [11]:
# Load the X variable from the test data
X_test = games_tourney_test['seed_diff']

# Load the y variable from the test data
y_test = games_tourney_test['score_diff']

# Evaluate the model on the test data
model.evaluate(X_test,y_test)

804/804 [==============================] - 0s 14us/step


9.520400165918455

In [12]:
games_season=pd.read_csv('E:\\PYTHON\\Data\\basketball\\games_season.csv')

In [13]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

In [14]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

In [15]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input((1,),name='Team-1-In')

# Separate input layer for team 2
team_in_2 = Input((1,),name='Team-2-In')

In [16]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

In [17]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

In [18]:
# Imports
from keras.layers import Subtract,Concatenate
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile('adam', 'mean_absolute_error')

In [19]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1,input_2],
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=0.1,
          verbose=True)

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
280960/280960 [==============================] - 0s 2us/step - loss: 12.1200 - val_loss: 11.8380


In [20]:
games_tourney=pd.read_csv('E:\\PYTHON\\Data\\basketball\\games_tourney.csv')

In [21]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
model.evaluate([input_1, input_2], games_tourney['score_diff'])

4234/4234 [==============================] - 0s 9us/step


11.680330610748276

In [22]:
# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

In [23]:
# Import the model class
from keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [24]:
# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=1,
          verbose=True,
          validation_split=0.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']],games_tourney['score_diff'])

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
4234/4234 [==============================] - 0s 11us/step


11.690033118906143

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Team-1-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-2-In (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
Team-Strength-Model (Model)     (None, 1)            10888       Team-1-In[0][0]                  
                                                                 Team-2-In[0][0]                  
__________________________________________________________________________________________________
Home-In (InputLayer)            (None, 1)            0                                            
__________

In [26]:
# Predict
games_tourney['pred'] = model.predict([games_tourney['team_1'],games_tourney['team_2'],games_tourney['home']])

In [27]:
# Create an input layer with 3 columns
input_tensor = Input((3,))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [28]:
games_tourney_train=games_tourney[games_tourney.season<2010]
games_tourney_test=games_tourney[games_tourney.season>=2010]

In [29]:
# Fit the model
model.fit(games_tourney_train[['home', 'seed_diff', 'pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)

Epoch 1/1
3168/3168 [==============================] - 0s 67us/step - loss: 15.8779


In [30]:
# Evaluate the model on the games_tourney_test dataset
model.evaluate(games_tourney_test[['home', 'seed_diff', 'pred']], 
               games_tourney_test['score_diff'])

1066/1066 [==============================] - 0s 51us/step


14.638439380652908

# Model woth 2 outputs

In [31]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam')

In [32]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=True,
  		  epochs=100,
  		  batch_size=16384)

Epoch 1/100
3168/3168 [==============================] - 0s 53us/step - loss: 71.9472
Epoch 2/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9460
Epoch 3/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9449
Epoch 4/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9438
Epoch 5/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9427
Epoch 6/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9416
Epoch 7/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9405
Epoch 8/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9393
Epoch 9/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9382
Epoch 10/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9371
Epoch 11/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.9359
Epoch 12/100
3168/3168 [==============================] - 0s 1

Epoch 97/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.8382
Epoch 98/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.8371
Epoch 99/100
3168/3168 [==============================] - 0s 1us/step - loss: 71.8359
Epoch 100/100
3168/3168 [==============================] - 0s 2us/step - loss: 71.8348


In [33]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[ 0.01538543,  0.57539284],
       [ 0.5532574 , -0.86861634]], dtype=float32), array([0.09996708, 0.09996708], dtype=float32)]
season        1997.045455
team_1        5546.025568
team_2        5546.025568
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.912247
score_2         71.912247
won              0.500000
pred             0.135529
dtype: float64


In [34]:
# Evaluate the model on the tournament test data
model.evaluate(games_tourney_test[['seed_diff', 'pred']],
  		  games_tourney_test[['score_1', 'score_2']])

1066/1066 [==============================] - 0s 70us/step


68.73198164605289

In [35]:
# Create an input layer with 2 columns
input_tensor = Input((2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = S=Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

In [36]:
# Import the Adam optimizer
from keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10
3168/3168 [==============================] - 0s 101us/step - loss: 16.0192 - dense_7_loss: 9.2096 - dense_8_loss: 6.8096
Epoch 2/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.9329 - dense_7_loss: 9.2160 - dense_8_loss: 6.7169
Epoch 3/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.8696 - dense_7_loss: 9.2224 - dense_8_loss: 6.6472
Epoch 4/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.7941 - dense_7_loss: 9.2289 - dense_8_loss: 6.5652
Epoch 5/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.7226 - dense_7_loss: 9.2354 - dense_8_loss: 6.4873
Epoch 6/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.6549 - dense_7_loss: 9.2428 - dense_8_loss: 6.4121
Epoch 7/10
3168/3168 [==============================] - 0s 1us/step - loss: 15.5900 - dense_7_loss: 9.2527 - dense_8_loss: 6.3373
Epoch 8/10
3168/3168 [==============================] - 0s 2us/step - loss: 15.5263 - de

In [37]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[ 0.95009285],
       [-0.1772691 ]], dtype=float32), array([[-1.2493427]], dtype=float32)]
season        1997.045455
team_1        5546.025568
team_2        5546.025568
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.912247
score_2         71.912247
won              0.500000
pred             0.135529
dtype: float64


In [38]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


In [39]:
# Evaluate the model on new data
model.evaluate(games_tourney_test[['seed_diff', 'pred']],
          [games_tourney_test[['score_diff']], games_tourney_test[['won']]])

1066/1066 [==============================] - 0s 107us/step


[14.683892273321383, 9.099845762771693, 5.584046520390609]